# Algoritmo de localización óptima de residencias

Objetivo: Seleccionar 1000 ubicaciones de residencias en España a partir de secciones censales, maximizando la demanda potencial y evitando solapamientos espaciales.

In [ ]:
import pandas as pd
import numpy as np
from math import radians
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import haversine_distances

In [ ]:
path = "../data/Datos caso práctico 2025 - renta y localización (def).xlsx"

df_loc = pd.read_excel(path, sheet_name="localizacion")
df_renta = pd.read_excel(path, sheet_name="renta")
df_sit = pd.read_excel(path, sheet_name="situacion")

df_loc.head()

df = df_loc.merge(df_renta, on="id_seccion", how="inner")
df = df.merge(df_sit, on="id_seccion", how="inner")
df = df.merge(df_pop, on="id_seccion", how="inner")

df.head()

In [ ]:
vars_pos = ["p55_65", "p80plus", "densidad"]
vars_neg = ["morosidad"]

vars_all = vars_pos + vars_neg

df = df.dropna(subset=vars_all + ["lat", "lon"])

scaler = MinMaxScaler()
df_norm = df.copy()

df_norm[vars_all] = scaler.fit_transform(df[vars_all])

# invertir las variables negativas
for v in vars_neg:
    df_norm[v] = 1 - df_norm[v]

In [ ]:
weights = {
    "p55_65": 0.15,
    "p80plus": 0.35,
    "morosidad": 0.25,
    "densidad": 0.25
}

df_norm["score"] = sum(
    df_norm[v] * w for v, w in weights.items()
)

In [ ]:
MIN_80 = 0.02  # ejemplo
df_filt = df_norm[df_norm["p80plus"] >= MIN_80].copy()

In [ ]:
df_cand = df_filt.sort_values("score", ascending=False).head(5000)

In [ ]:
def haversine_km(p1, p2):
    return haversine_distances(
        [[radians(p1[0]), radians(p1[1])]],
        [[radians(p2[0]), radians(p2[1])]]
    )[0][0] * 6371

In [ ]:
def select_locations(df, k=1000, D=3):
    selected = []
    coords_selected = []

    for _, row in df.iterrows():
        if len(selected) >= k:
            break

        point = (row["lat"], row["lon"])

        if all(haversine_km(point, c) >= D for c in coords_selected):
            selected.append(row)
            coords_selected.append(point)

    return pd.DataFrame(selected)

In [ ]:
D = 3
for _ in range(20):
    res = select_locations(df_cand, k=1000, D=D)
    if len(res) < 1000:
        D *= 0.9
    elif len(res) > 1000:
        D *= 1.1

In [ ]:
len(res)
res.head()

import folium

m = folium.Map(location=[40, -3], zoom_start=6)
for _, r in res.iterrows():
    folium.CircleMarker(
        [r["lat"], r["lon"]],
        radius=3,
        color="red"
    ).add_to(m)
m